In [ ]:
%matplotlib widget
import plopp
import ipywidgets as ipw
import scipp as sc
from dataclasses import dataclass
from numpy.random import default_rng

rng = default_rng(seed=1234)


def make_random_data() -> sc.DataArray:
    nrow = 1000
    x = sc.array(dims=['row'], unit='m', values=(rng.random(nrow)))
    y = sc.array(dims=['row'], unit='m', values=(rng.random(nrow)))
    data = sc.ones(dims=['row'], unit='K', shape=[nrow])
    data.values += 0.1 * rng.random(nrow)
    return sc.DataArray(data=data, coords={'x': x, 'y': y}).hist(x=10, y=10)

In [ ]:
import threading
import time


@dataclass
class DataHandle:
    value: sc.DataArray


class DataProvider:
    def __init__(self, value: sc.DataArray):
        self._handle = DataHandle(value)
        self._node = plopp.Node(self._handle)

        @plopp.node
        def get_value(handle: DataHandle) -> sc.DataArray:
            return handle.value

        self.value_node = get_value(self._node)

    def update_data(self, value: sc.DataArray):
        self._handle.value = value
        self._node.notify_children(message='hi')


da = make_random_data()

provider = DataProvider(da)


def run_change_data():
    for _ in range(100):
        provider.update_data(make_random_data())
        time.sleep(0.1)


thread = threading.Thread(target=run_change_data)
thread.start()

image = plopp.imagefigure(provider.value_node, norm='linear', vmin=1, vmax=100)
ipw.VBox([image])